<a href="https://colab.research.google.com/github/creamburger/TensorStudy/blob/main/3%EA%B0%95_%EC%A6%89%EC%8B%9C_%EC%8B%A4%ED%96%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

In [5]:
x = [[2.]]
m = tf.matmul(x, x)
m
print("hello, {}".format(m))

hello, [[4.]]


In [6]:
a = tf.constant([[1,2], [3,4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [7]:
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [8]:
print(a * b) #성분곱셈(각각의 행렬의 값을 그 자리의 값끼리 곱하는 것)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [9]:
print(a @ b) #행렬곱

tf.Tensor(
[[10 13]
 [22 29]], shape=(2, 2), dtype=int32)


In [10]:
tf.multiply(a, b) #(성분곱)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 2,  6],
       [12, 20]], dtype=int32)>

In [11]:
tf.matmul(a, b) #(행렬곱)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10, 13],
       [22, 29]], dtype=int32)>

In [12]:
print(a.numpy()) #a에 있는 값을 numpy로 변형

[[1 2]
 [3 4]]


In [13]:
def fizzbuzz(max_num):
  counter = tf.constant(0)
  max_num = tf.convert_to_tensor(max_num)
  for num in range(1, max_num.numpy()+1):
    num = tf.constant(num)
    if int(num % 3) == 0 and int(num % 5) == 0:
      print('FizzBuzz')
    elif int(num % 3) == 0:
      print('Fizz')
    elif int(num % 5) == 0:
      print('Buzz')
    else:
      print(num.numpy())
    counter += 1

In [14]:
fizzbuzz(33)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
Fizz
22
23
Fizz
Buzz
26
Fizz
28
29
FizzBuzz
31
32
Fizz


In [16]:
w = tf.Variable([[1.0]])  #w가 loss 값에 대해 얼마나 영향을 미쳤는지 구하는 코드(미분식)
with tf.GradientTape() as tape:
  loss = w * w # w ** 2 미분하면 2w
grad = tape.gradient(loss, w)
print(grad) # 2. #첫 번째 줄에서 tf.Variable로 w값을 1로 정의했으므로

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


In [17]:
for x in zip([1,2,3], [4,5,6]):
  print(x) #zip이라는 연산자는 두 개씩 꺼내어 사용할 수 있게 만들어줌

(1, 4)
(2, 5)
(3, 6)


In [18]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2926 - accuracy: 0.9149
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1420 - accuracy: 0.9576
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1067 - accuracy: 0.9670
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0886 - accuracy: 0.9722
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0748 - accuracy: 0.9765
313/313 - 0s - loss: 0.0720 - accuracy: 0.9773 - 497ms/epoch - 2ms/step


[0.0719529315829277, 0.9772999882698059]

In [19]:
# mnist 데이터 가져오기 및 포맷 맞추기
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
   tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

# 모델 생성
mnist_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu',
                         input_shape=(None, None, 1)),
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])
for images,labels in dataset.take(1):
  print("로짓: ", mnist_model(images[0:1]).numpy())


optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_history = []

def train_step(images, labels):
  with tf.GradientTape() as tape:
    logits = mnist_model(images, training=True)
   
    # 결과의 형태를 확인하기 위해서 단언문 추가
    tf.debugging.assert_equal(logits.shape, (32, 10))
   
    loss_value = loss_object(labels, logits)

  loss_history.append(loss_value.numpy().mean())
  grads = tape.gradient(loss_value, mnist_model.trainable_variables)
  optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))
  # W = W - alpha(learning rate) * gradients
  # dL/dW: gradients

def train():
  for epoch in range(3):
    for (batch, (images, labels)) in enumerate(dataset):
      train_step(images, labels)
    print ('에포크 {} 종료'.format(epoch))

train()

로짓:  [[-0.01285741 -0.00469505  0.01222313  0.00349638 -0.00951262 -0.00523054
   0.01927401 -0.04009932 -0.02169677  0.00208157]]
에포크 0 종료
에포크 1 종료
에포크 2 종료


In [22]:
import matplotlib.pyplot as plt
plt.plot(training_inputs, training_outputs)

NameError: ignored

In [23]:
class Model(tf.keras.Model):
  def __init__(self):
    super(Model, self).__init__()
    self.W = tf.Variable(5., name='weight')
    self.B = tf.Variable(10., name='bias')
  def call(self, inputs):
    return inputs * self.W + self.B

In [25]:
class Model(tf.keras.Model):
  def __init__(self):
    super(Model, self).__init__()
    self.W = tf.Variable(5., name='weight')
    self.B = tf.Variable(10., name='bias')
  def call(self, inputs):
    return inputs * self.W + self.B
  # 약 3 * x + 2개의 점으로 구성된 실험 데이터
NUM_EXAMPLES = 2000
training_inputs = tf.random.normal([NUM_EXAMPLES])
noise = tf.random.normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise
# 최적화할 손실함수
def loss(model, inputs, targets):
  error = model(inputs) - targets
  return tf.reduce_mean(tf.square(error)) # MSE, mean squared error

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, [model.W, model.B])
# 정의:
# 1. 모델
# 2. 모델 파라미터에 대한 손실 함수의 미분
# 3. 미분에 기초한 변수 업데이트 전략
model = Model()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

print("초기 손실: {:.3f}".format(loss(model, training_inputs, training_outputs)))

초기 손실: 69.654


In [26]:
# 반복 훈련
for i in range(300):
  grads = grad(model, training_inputs, training_outputs)
  optimizer.apply_gradients(zip(grads, [model.W, model.B]))
  if i % 20 == 0:
    print("스텝 {:03d}에서 손실: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))
    # print(grads[0].numpy(), grads[1].numpy(), model.W.numpy(), model.B.numpy())

print("최종 손실: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

스텝 000에서 손실: 66.923
스텝 020에서 손실: 30.268
스텝 040에서 손실: 13.993
스텝 060에서 손실: 6.767
스텝 080에서 손실: 3.558
스텝 100에서 손실: 2.133
스텝 120에서 손실: 1.500
스텝 140에서 손실: 1.219
스텝 160에서 손실: 1.095
스텝 180에서 손실: 1.039
스텝 200에서 손실: 1.015
스텝 220에서 손실: 1.004
스텝 240에서 손실: 0.999
스텝 260에서 손실: 0.997
스텝 280에서 손실: 0.996
최종 손실: 0.995
W = 2.9808948040008545, B = 2.002577304840088
